In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

Process and Intake Data

In [ ]:
def preprocess_data():
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
    
    # Convert to PyTorch tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)
    
    return X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor, scaler



Simple MLP Model

In [ ]:
class StockPredictor(nn.Module):
    def __init__(self, input_size, hidden_size=64, output_size=1):
        super(StockPredictor, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size)
        )
    
    def forward(self, x):
        return self.model(x)

Training & Model Evaluation

In [ ]:
def train_model(model, criterion, optimizer, X_train, y_train, num_epochs=100):
 
    for epoch in range(num_epochs):
        # Forward pass
        predictions = model(X_train)
        loss = criterion(predictions, y_train)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (epoch + 1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")
    return model


def evaluate_model_mape(model, X_test, y_test):

    model.eval() 
    with torch.no_grad():
        predictions = model(X_test)
        absolute_percentage_error = torch.abs((y_test - predictions) / y_test) * 100
        mape = torch.mean(absolute_percentage_error)
        
        print(f"MAPE: {mape.item():.4f}%")
        return predictions, mape


Hyperparater Tuning & Main

In [ ]:
if __name__ == "__main__":
    X_train, X_test, y_train, y_test, scaler = preprocess_data()
    
    # Model hyperparameters
    input_size = X_train.shape[1]
    hidden_size = 64
    output_size = 1
    learning_rate = 0.001
    num_epochs = 100
    
    model = StockPredictor(input_size, hidden_size, output_size)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    trained_model = train_model(model, criterion, optimizer, X_train, y_train, num_epochs)
    
    predictions, test_loss = evaluate_model(trained_model, criterion, X_test, y_test)
